# EuroParl Analysis

Goal: count how much each speaker in the European Parliament was speaking.

The [EuroParl website](http://www.europarl.europa.eu/plenary/en/debates-video.html#sidesForm) where one can enter a search query and a [sample debate entry](http://www.europarl.europa.eu/sides/getDoc.do?pubRef=-//EP//TEXT+CRE+20180416+ITEM-021+DOC+XML+V0//EN&language=EN&query=INTERV&detail=1-107-000).

In [59]:
import urllib
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
r = requests.post("http://www.europarl.europa.eu/plenary/en/debates-video.html",
    data='source=&clean=false&legChange=false&leg=8&refSittingDateStart=&refSittingDateEnd=&oraMep=Dubravka+%C5%A0UICA&oraNoMep=&miType=title&miText=',
    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate",
        "Accept-Language": "hr,en-GB;q=0.9,en;q=0.8,de;q=0.7,bs;q=0.6",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Content-Type": "application/x-www-form-urlencoded",
        "DNT": "1",
        "Origin": "http://www.europarl.europa.eu",
        "Referer": "http://www.europarl.europa.eu/plenary/en/debates-video.html",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"
    },
    cookies={
        "JSESSIONID": "44774A7FF42ED8AD7A0DA5D81C90574D.node2",
        "WT_FPC": "id=2068e61a29c18f8c3fb1551540300172:lv=1551544544639:ss=1551544544639",
        "__verify": "1",
        "crid": "node2",
        "europarl_cookies_accepted": "1",
        "europarlcookiepolicysagreement": "1"
    },
)
r.content

b'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" prefix="og:  http://ogp.me/ns#" lang="en">\r\n\r\n\r\n\r\n\r\n<head>\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n\t\r\n\t\t\r\n\t\r\n\t\r\n\t<title>Debates and Videos | Plenary | European Parliament</title>\r\n\t<meta name="title" content="Debates and Videos | Plenary | European Parliament"/>\t\r\n\t<meta property="og:title" content="Debates and Videos | Plenary | European Parliament"/>\r\n\t\r\n\t<meta property="og:description" content="Extracts and multimedia per debate such as verbatim reports, videos of speakers, texts of interventions, material to download. Search multimedia by speaker and date."/>\r\n\t<meta name="description" content="Extracts and multimedia per debate such as verbatim reports, videos of speakers, texts of interventions, m

In [3]:
soup = BeautifulSoup(r.content, 'html.parser')
int(soup.select('.results')[0].contents[0].split(' : ')[1])

1128

Make it a function

In [126]:
def get_speaker_results(speaker, term=8, start=datetime(2018, 1, 1), end=datetime(2018, 12, 31)):
    refSittingDateStart = start.strftime('%d/%m/%Y')
    refSittingDateEnd = end.strftime('%d/%m/%Y')
    data = urllib.parse.urlencode({
        'source': '',
        'clean': 'false',
        'legChange': 'false',
        'leg': 8,
        'refSittingDateStart': refSittingDateStart,
        'refSittingDateEnd': refSittingDateEnd,
        'oraMep': speaker,
        'oraNoMep': '',
        'miType': 'title',
        'miText': ''
    })

    r = requests.post("http://www.europarl.europa.eu/plenary/en/debates-video.html",
        data=data,
        headers={
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
            "Accept-Encoding": "gzip, deflate",
            "Accept-Language": "hr,en-GB;q=0.9,en;q=0.8,de;q=0.7,bs;q=0.6",
            "Cache-Control": "max-age=0",
            "Connection": "keep-alive",
            "Content-Type": "application/x-www-form-urlencoded",
            "DNT": "1",
            "Origin": "http://www.europarl.europa.eu",
            "Referer": "http://www.europarl.europa.eu/plenary/en/debates-video.html",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"
        },
    )

    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        results = int(soup.select('.results')[0].contents[0].split(' : ')[1])
    except (IndexError, ValueError):
        results = 0
    return results

get_speaker_results('Davor ŠKRLEC')


11

Get full list of MEPs

In [20]:
ls ./data/MEPs

a.xml     e.xml     h.xml     l.xml     p.xml     t.xml     x.xml
b.xml     f.xml     i.xml     m.xml     q.xml     u.xml     y.xml
c.xml     full.xml  j.xml     n.xml     r.xml     v.xml     z.xml
d.xml     g.xml     k.xml     o.xml     s.xml     w.xml


In [119]:
import xml.etree.ElementTree as ET
tree = ET.parse('./data/MEPs/full.xml')
root = tree.getroot()
mep = list(root)[0]

def parse_mep(mep):    
    speaker = mep.find('fullName').text
    country = mep.find('country').text
    politicalGroup = mep.find('politicalGroup').text
    nationalPoliticalGroup = mep.find('nationalPoliticalGroup').text
    return {'speaker': speaker, 'country': country, 'politicalGroup': politicalGroup, 'nationalPoliticalGroup': nationalPoliticalGroup}

parse_mep(mep)

{'speaker': 'Asim ADEMOV',
 'country': 'Bulgaria',
 'politicalGroup': "Group of the European People's Party (Christian Democrats)",
 'nationalPoliticalGroup': 'Citizens for European Development of Bulgaria'}

In [112]:
starts = pd.date_range('2014-07-01', '2019-02-28', freq='MS')
starts

DatetimeIndex(['2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01',
               '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01',
               '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01',
               '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01',
               '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01',
               '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01',
               '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01',
               '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01',
               '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01',
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01',
               '2018-07-01', '2018-08-01', '2018-09-01', '2018-10-01',
               '2018-11-01', '2018-12-01', '2019-01-01', '2019-02-01'],
     

In [113]:
ends = pd.date_range('2014-07-01', '2019-02-28', freq='M')
ends

DatetimeIndex(['2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31',
               '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28',
               '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30',
               '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31',
               '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29',
               '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30',
               '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31',
               '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28',
               '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30',
               '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31',
               '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28',
               '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
               '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31',
               '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28'],
     

In [123]:
# speaker_results = {}
# for start, end in list(zip(starts, ends)): # [:5]:
#     monthly_speaker_results = {}
#     speaker_results[start.date()] = monthly_speaker_results
#     print(start.date(), end.date())
#     for mep in list(root): # [:5]:
#         # speaker = mep.find('fullName').text
#         speaker_dict = parse_mep(mep)
#         speaker = speaker_dict['speaker']
#         print(speaker)
# #         start = datetime.strptime('2018-01-01', '%Y-%m-%d')
# #         end = datetime.strptime('2018-12-31', '%Y-%m-%d')
#         results = get_speaker_results(speaker, start=start, end=end)
#         monthly_speaker_results[speaker] = results
# speaker_results = pd.DataFrame(speaker_results)
# speaker_results.to_csv('./data/europarl.csv')
# speaker_results

2014-07-01 2014-07-31
Asim ADEMOV
Isabella ADINOLFI
Marco AFFRONTE
Laura AGEA
John Stuart AGNEW
Clara Eugenia AGUILERA GARCÍA
Daniela AIUTO
Tim AKER
Marina ALBIOL GUZMÁN
Nedzhmi ALI
Michèle ALLIOT-MARIE
Lucy ANDERSON
Martina ANDERSON
Max ANDERSSON
Eric ANDRIEU
Laima Liucija ANDRIKIENĖ
Nikos ANDROULAKIS
Gerolf ANNEMANS
Maria ARENA
Pascal ARIMONT
Marie-Christine ARNAUTU
Jonathan ARNOTT
Jean ARTHUIS
Richard ASHWORTH
Francisco ASSIS
Janice ATKINSON
Margrete AUKEN
Petras AUŠTREVIČIUS
Inés AYALA SENDER
Pilar AYUSO
Johannes Cornelis van BAALEN
Georges BACH
Guillaume BALAS
Zigmantas BALČYTIS
Zoltán BALCZÓ
Nikolay BAREKOV
Amjad BASHIR
Gerard BATTEN
Nicolas BAY
Hugues BAYET
Catherine BEARDER
Beatriz BECERRA BASTERRECHEA
Heinz K. BECKER
Tiziana BEGHIN
Bas BELDER
Ivo BELET
Bendt BENDTSEN
Brando BENIFEI
Xabier BENITO ZILUAGA
Monika BEŇOVÁ
Pervenche BERÈS
Joëlle BERGERON
Goffredo Maria BETTINI
Izaskun BILBAO BARANDICA
Dominique BILDE
Mara BIZZOTTO
Malin BJÖRK
José BLANCO LÓPEZ
Vilija BLINKEVIČIŪTĖ
A

Margot PARKER
Ioan Mircea PAŞCU
Aldo PATRICIELLO
Emilian PAVEL
Jiří PAYNE
Piernicola PEDICINI
Vincent PEILLON
Alojz PETERLE
Morten Helveg PETERSEN
Marijana PETIR
Florian PHILIPPOT
Pina PICIERNO
Tonino PICULA
Bolesław G. PIECHA
Markus PIEPER
Sirpa PIETIKÄINEN
João PIMENTA LOPES
Mirosław PIOTROWSKI
Kati PIRI
Georgi PIRINSKI
Julia PITERA
Marek PLURA
Pavel POC
Miroslav POCHE
Stanislav POLČÁK
Maurice PONGA
Răzvan POPA
Tomasz Piotr PORĘBA
Jiří POSPÍŠIL
Soraya POST
Cristian Dan PREDA
Marcus PRETZELL
Gabriele PREUSS
John PROCTER
Franck PROUST
Carolina PUNSET
Godelieve QUISTHOUDT-ROWOHL
Emil RADEV
Jozo RADOŠ
Dennis RADTKE
Paulo RANGEL
Laurenţiu REBEGA
Julia REDA
Evelyn REGNER
Julia REID
Michel REIMON
Terry REINTKE
Christine REVAULT D'ALLONNES BONNEFOY
Sofia RIBEIRO
Frédérique RIES
Dominique RIQUET
Michèle RIVASI
Robert ROCHEFORT
Liliana RODRIGUES
Maria João RODRIGUES
Inmaculada RODRÍGUEZ-PIÑERO FERNÁNDEZ
Ulrike RODUST
Jens ROHDE
Claude ROLIN
Bronis ROPĖ
Dariusz ROSATI
Virginie ROZIÈRE
Fernando 

Takis HADJIGEORGIOU
Jussi HALLA-AHO
Thomas HÄNDEL
Daniel HANNAN
Christophe HANSEN
Marian HARKIN
Rebecca HARMS
Martin HÄUSLING
Heidi HAUTALA
Brian HAYES
Anja HAZEKAMP
Anna HEDH
Hans-Olaf HENKEL
Esther HERRANZ GARCÍA
Krzysztof HETMAN
Maria HEUBUCH
Nadja HIRSCH
Czesław HOC
Iris HOFFMANN
Monika HOHLMEIER
Gunnar HÖKMARK
György HÖLVÉNYI
Mary HONEYBALL
Mike HOOKEM
Brice HORTEFEUX
John HOWARTH
Danuta Maria HÜBNER
Ian HUDGHTON
Jan HUITEMA
Filiz HYUSMENOVA
Sophia in 't VELD
Carlos ITURGAIZ
Cătălin Sorin IVAN
Robert Jarosław IWASZKIEWICZ
Liisa JAAKONSAARI
Anneli JÄÄTTEENMÄKI
Yannick JADOT
Peter JAHR
Ivan JAKOVČIĆ
Jean-François JALKH
Diane JAMES
France JAMET
Lívia JÁRÓKA
Ramón JÁUREGUI ATONDO
Benedek JÁVOR
Danuta JAZŁOWIECKA
Petr JEŽEK
Teresa JIMÉNEZ-BECERRIL BARRIO
Eva JOLY
Dennis de JONG
Agnes JONGERIUS
Marc JOULAUD
Marek JUREK
Philippe JUVIN
Karin KADENBACH
Eva KAILI
Jarosław KALINOWSKI
Sandra KALNIETE
Syed KAMALL
Petra KAMMEREVERT
Barbara KAPPEL
Othmar KARAS
Rina Ronja KARI
Sajjad KARIM
Rikke-

Clara Eugenia AGUILERA GARCÍA
Daniela AIUTO
Tim AKER
Marina ALBIOL GUZMÁN
Nedzhmi ALI
Michèle ALLIOT-MARIE
Lucy ANDERSON
Martina ANDERSON
Max ANDERSSON
Eric ANDRIEU
Laima Liucija ANDRIKIENĖ
Nikos ANDROULAKIS
Gerolf ANNEMANS
Maria ARENA
Pascal ARIMONT
Marie-Christine ARNAUTU
Jonathan ARNOTT
Jean ARTHUIS
Richard ASHWORTH
Francisco ASSIS
Janice ATKINSON
Margrete AUKEN
Petras AUŠTREVIČIUS
Inés AYALA SENDER
Pilar AYUSO
Johannes Cornelis van BAALEN
Georges BACH
Guillaume BALAS
Zigmantas BALČYTIS
Zoltán BALCZÓ
Nikolay BAREKOV
Amjad BASHIR
Gerard BATTEN
Nicolas BAY
Hugues BAYET
Catherine BEARDER
Beatriz BECERRA BASTERRECHEA
Heinz K. BECKER
Tiziana BEGHIN
Bas BELDER
Ivo BELET
Bendt BENDTSEN
Brando BENIFEI
Xabier BENITO ZILUAGA
Monika BEŇOVÁ
Pervenche BERÈS
Joëlle BERGERON
Goffredo Maria BETTINI
Izaskun BILBAO BARANDICA
Dominique BILDE
Mara BIZZOTTO
Malin BJÖRK
José BLANCO LÓPEZ
Vilija BLINKEVIČIŪTĖ
Andrea BOCSKOR
Reimer BÖGE
Franc BOGOVIČ
Simona BONAFÈ
Michał BONI
Mario BORGHEZIO
David BORRELLI

Piernicola PEDICINI
Vincent PEILLON
Alojz PETERLE
Morten Helveg PETERSEN
Marijana PETIR
Florian PHILIPPOT
Pina PICIERNO
Tonino PICULA
Bolesław G. PIECHA
Markus PIEPER
Sirpa PIETIKÄINEN
João PIMENTA LOPES
Mirosław PIOTROWSKI
Kati PIRI
Georgi PIRINSKI
Julia PITERA
Marek PLURA
Pavel POC
Miroslav POCHE
Stanislav POLČÁK
Maurice PONGA
Răzvan POPA
Tomasz Piotr PORĘBA
Jiří POSPÍŠIL
Soraya POST
Cristian Dan PREDA
Marcus PRETZELL
Gabriele PREUSS
John PROCTER
Franck PROUST
Carolina PUNSET
Godelieve QUISTHOUDT-ROWOHL
Emil RADEV
Jozo RADOŠ
Dennis RADTKE
Paulo RANGEL
Laurenţiu REBEGA
Julia REDA
Evelyn REGNER
Julia REID
Michel REIMON
Terry REINTKE
Christine REVAULT D'ALLONNES BONNEFOY
Sofia RIBEIRO
Frédérique RIES
Dominique RIQUET
Michèle RIVASI
Robert ROCHEFORT
Liliana RODRIGUES
Maria João RODRIGUES
Inmaculada RODRÍGUEZ-PIÑERO FERNÁNDEZ
Ulrike RODUST
Jens ROHDE
Claude ROLIN
Bronis ROPĖ
Dariusz ROSATI
Virginie ROZIÈRE
Fernando RUAS
Paul RÜBIG
Pirkko RUOHONEN-LERNER
Kārlis ŠADURSKIS
Tokia SAÏFI
Sofia 

Daniel HANNAN
Christophe HANSEN
Marian HARKIN
Rebecca HARMS
Martin HÄUSLING
Heidi HAUTALA
Brian HAYES
Anja HAZEKAMP
Anna HEDH
Hans-Olaf HENKEL
Esther HERRANZ GARCÍA
Krzysztof HETMAN
Maria HEUBUCH
Nadja HIRSCH
Czesław HOC
Iris HOFFMANN
Monika HOHLMEIER
Gunnar HÖKMARK
György HÖLVÉNYI
Mary HONEYBALL
Mike HOOKEM
Brice HORTEFEUX
John HOWARTH
Danuta Maria HÜBNER
Ian HUDGHTON
Jan HUITEMA
Filiz HYUSMENOVA
Sophia in 't VELD
Carlos ITURGAIZ
Cătălin Sorin IVAN
Robert Jarosław IWASZKIEWICZ
Liisa JAAKONSAARI
Anneli JÄÄTTEENMÄKI
Yannick JADOT
Peter JAHR
Ivan JAKOVČIĆ
Jean-François JALKH
Diane JAMES
France JAMET
Lívia JÁRÓKA
Ramón JÁUREGUI ATONDO
Benedek JÁVOR
Danuta JAZŁOWIECKA
Petr JEŽEK
Teresa JIMÉNEZ-BECERRIL BARRIO
Eva JOLY
Dennis de JONG
Agnes JONGERIUS
Marc JOULAUD
Marek JUREK
Philippe JUVIN
Karin KADENBACH
Eva KAILI
Jarosław KALINOWSKI
Sandra KALNIETE
Syed KAMALL
Petra KAMMEREVERT
Barbara KAPPEL
Othmar KARAS
Rina Ronja KARI
Sajjad KARIM
Rikke-Louise KARLSSON
Karol KARSKI
Elsi KATAINEN
Sylvia-

Tim AKER
Marina ALBIOL GUZMÁN
Nedzhmi ALI
Michèle ALLIOT-MARIE
Lucy ANDERSON
Martina ANDERSON
Max ANDERSSON
Eric ANDRIEU
Laima Liucija ANDRIKIENĖ
Nikos ANDROULAKIS
Gerolf ANNEMANS
Maria ARENA
Pascal ARIMONT
Marie-Christine ARNAUTU
Jonathan ARNOTT
Jean ARTHUIS
Richard ASHWORTH
Francisco ASSIS
Janice ATKINSON
Margrete AUKEN
Petras AUŠTREVIČIUS
Inés AYALA SENDER
Pilar AYUSO
Johannes Cornelis van BAALEN
Georges BACH
Guillaume BALAS
Zigmantas BALČYTIS
Zoltán BALCZÓ
Nikolay BAREKOV
Amjad BASHIR
Gerard BATTEN
Nicolas BAY
Hugues BAYET
Catherine BEARDER
Beatriz BECERRA BASTERRECHEA
Heinz K. BECKER
Tiziana BEGHIN
Bas BELDER
Ivo BELET
Bendt BENDTSEN
Brando BENIFEI
Xabier BENITO ZILUAGA
Monika BEŇOVÁ
Pervenche BERÈS
Joëlle BERGERON
Goffredo Maria BETTINI
Izaskun BILBAO BARANDICA
Dominique BILDE
Mara BIZZOTTO
Malin BJÖRK
José BLANCO LÓPEZ
Vilija BLINKEVIČIŪTĖ
Andrea BOCSKOR
Reimer BÖGE
Franc BOGOVIČ
Simona BONAFÈ
Michał BONI
Mario BORGHEZIO
David BORRELLI
Biljana BORZAN
Victor BOŞTINARU
Louise BOUR

ValueError: invalid literal for int() with base 10: ''

In [137]:
# pd.DataFrame(speaker_results).transpose()[:4].astype(int).to_csv('./data/europarl.csv')

In [29]:
mep = list(root)[0]

In [34]:
fullName = mep[0] #['fullName']
fullName.text

'Uma AALTONEN'

In [66]:
start = datetime.strptime('2018-01-01', '%Y-%m-%d')

,2014-07-01,2014-08-01
Damien ABAD,127,0
Uma AALTONEN,127,0
